# <center> <span style="color:GREEN"> COMMANDS SYSTEM USING CNN

In [2]:
import numpy as np                        # linear algebra library
import pandas as pd                       # data frames processing
import matplotlib.pyplot as plt           # visualization library
import seaborn as sn                      # visualization library
import math
import keras
from sys import byteorder
from array import array
from struct import pack




# Audio processing libraries
import scipy.io.wavfile as wav
import IPython.display as ipd
import librosa
import librosa.display
import scipy.signal as signal
import noisereduce as nr
from IPython.display import Audio, IFrame, display
from scipy.signal import hilbert
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv
import soundfile as sf
import malaya_speech
import webrtcvad
from webrtcvad import Vad
from malaya_speech import Pipeline
import speech_recognition as spreg
import pyaudio
import wave
import threading
from python_speech_features import mfcc

#simulation library
import pygame



C:\Users\Esraa\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
C:\Users\Esraa\Anaconda3\lib\site-packages\malaya_boilerplate\frozen_graph.py:35: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  'Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0'


pygame 2.1.2 (SDL 2.0.18, Python 3.7.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
model = keras.models.load_model('commands_CNN.h5')    #loading model

In [4]:
classes = ["down","left","right","stop","up"]    #output classes 

In [5]:
sr=16000                                        # sampling rate

In [6]:
def audio_preprocessing(wave):
    """ this function takes file.wav and process it through the following steps:
        1. reduce bakground noise.
        2. silence removal"spectral gating".   ################
        3. padding with zeros to have a constant vector length.
     Input: audio_file.wav
     Output: sampled signal"""
    samples = nr.reduce_noise(y=wave, sr=sr,stationary=True)
    vad = malaya_speech.vad.webrtc()
    y_= malaya_speech.resample(samples, sr, 16000)
    y_ = malaya_speech.astype.float_to_int(y_)
    frames = malaya_speech.generator.frames(samples, 30, sr)
    frames_ = list(malaya_speech.generator.frames(y_, 30, 16000, append_ending_trail = False))
    frames_webrtc = [(frames[no], vad(frame)) for no, frame in enumerate(frames_)]
    y_ = malaya_speech.combine.without_silent(frames_webrtc)
    if len(y_)>=19999:
            y_=y_[:19999]
    size = (1*sr+4000)-y_.shape[0]
    zero = np.zeros(size)
    signal = np.concatenate((y_,zero))
    return signal

In [7]:
def MFCCs(signal):
    """this function extract MFCCs from samples signal"""
    mfcc_feat = mfcc(signal, sr, winlen=256/sr, winstep=256/(2*sr), numcep=13, nfilt=26, nfft=256,
                 lowfreq=0, highfreq=sr/2, preemph=0.97, ceplifter=22, appendEnergy=True, winfunc=np.hamming)
    audio = np.transpose(mfcc_feat)
    return audio

In [8]:
THRESHOLD = 1200          # silence threshold
CHUNK_SIZE = 1024         # frame size
FORMAT = pyaudio.paInt16
RATE = 16000              # sample rate

def is_silent(snd_data):
    "Returns 'True' if below the 'silent' threshold"
    return max(snd_data) < THRESHOLD

def normalize(snd_data):
    "Average the volume out"
    MAXIMUM = 16384
    times = float(MAXIMUM)/max(abs(i) for i in snd_data)

    r = array('h')
    for i in snd_data:
        r.append(int(i*times))
    return r


def record():
    """
    Record a word or words from the microphone and 
    return the data as an array of signed shorts.

    Normalizes the audio, trims silence from the 
    start and end, and pads with 0.128 second of 
    blank sound to make sure VLC et al can play 
    it without getting chopped off.
    """
    p = pyaudio.PyAudio()
    stream = p.open(format=FORMAT, channels=1, rate=RATE,
        input=True, output=True,
        frames_per_buffer=CHUNK_SIZE)

    num_silent = 0
    snd_started = False

    r = array('h')

    while 1:
        # little endian, signed short
        snd_data = array('h', stream.read(CHUNK_SIZE))
        if byteorder == 'big':
            snd_data.byteswap()
        r.extend(snd_data)

        silent = is_silent(snd_data)

        if silent and snd_started:
            num_silent += 1
        elif not silent and not snd_started:
            snd_started = True

        if snd_started and num_silent > 2:
            break
        

    sample_width = p.get_sample_size(FORMAT)
    stream.stop_stream()
    stream.close()
    p.terminate()

    r = normalize(r)
    return sample_width, r

def record_to_file(path):
    "Records from the microphone and outputs the resulting data as output file"
    sample_width, data = record()
    data = pack('<' + ('h'*len(data)), *data)

    wf = wave.open(path, 'wb')
    wf.setnchannels(1)
    wf.setsampwidth(sample_width)
    wf.setframerate(RATE)
    wf.writeframes(data)
    wf.close()

In [9]:
def play_audio(sound_file):
    """this function plays audio in jupyter notebook"""
    wave, sr = sf.read(sound_file, dtype='float32')  
    sd.play(wave, sr)
    status = sd.wait()  # Wait until file is done playing
#play_audio(wave,sr)

In [10]:
def plot_audio(sound_file):
    """this function plots audio wave in jupyter notebook """
    wave, sr = sf.read(sound_file, dtype='float32')  
    samples = wave
    time = np.linspace(0, len(samples - 1) / sr, len(samples - 1))
    plt.plot(time,samples)  
    plt.title("Voice Signal")
    plt.xlabel("Time [seconds]")
    plt.ylabel("Voice amplitude")
    plt.show()
#plot_audio(y_)

In [11]:
def pred_func(sound_file):
    """this function takes sound file and call processing function to 
    adjust it then pass it to the MFCC function to be converted  then 
    passed to the model for prediction."""
    wave, sr = sf.read(sound_file, dtype='float32')  
    fs = sr
    signal=audio_preprocessing(wave)
    audio=MFCCs(signal)
    d1 = np.array(audio.shape[0])
    d2 = np.array(audio.shape[1])
    d = d1*d2
    r = []
    r = model.predict(audio.reshape(1,d1,d2,-1))
    ind = np.argmax(r)
    output=classes[ind]
    print('**** predicted output is: ',classes[ind], '****')
    return output

In [12]:
def run_func():
    """this fuction runs in the background continously to detect audio"""
    while True:
        global o_p
        print("please speak a word into the microphone")
        record_to_file('demo.wav')
        sound_file="demo.wav"
        o_p=pred_func(sound_file) 

In [13]:
game=threading.Thread(target=run_func)      # intialize thread to run in the background
game.start()

please speak a word into the microphone


## Implementaion of the Game Simulation

In [15]:
# activate the pygame library .
# initiate pygame and give permission
# to use pygame's functionality.
pygame.init()

# create the display surface object of specific dimension..e(500, 500).
win = pygame.display.set_mode((500, 500))

# set the pygame window name
pygame.display.set_caption("Moving rectangle")

# object current co-ordinates
x = 200
y = 200

# dimensions of the object
width = 20
height = 20

# velocity / speed of movement
vel = 0.02
i=0
o_p=''
# Indicates pygame is running
run = True
# infinite loop
while run:
    # iterate over the list of Event objects that was returned by pygame.event.get() method.
    for event in pygame.event.get():

        # if event object type is QUIT then quitting the pygame and program both.
        if event.type == pygame.QUIT:

            # it will make exit the while loop
            run = False
        
    # if left word is detected
    if o_p =='left' and x>0:
        x -= vel          # decrement in x co-ordinate


    # if right word is detected
    if o_p =='right' and x<500-width:
        x += vel          # increment in x co-ordinate


    # if up word is detected
    if o_p =='up' and y>0:
        y -= vel         # decrement in y co-ordinate


    # if down word is detected
    if o_p =='down' and y<500-height:
        y += vel            # increment in y co-ordinate
   
    # if stop word is detected
    if o_p =='stop' :
        y = y            # freeze in same position
        x = x
       
    # completely fill the surface object with black colour
    win.fill((0, 0, 0))

    # drawing object on screen which is rectangle 
    pygame.draw.rect(win, (255, 0, 0), (x, y, width, height))

    # it refreshes the window
    pygame.display.update()
    
# closes the pygame window
pygame.quit()
